In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)

from urllib.request import Request, urlopen
import json

from pprint import *
pp = PrettyPrinter(indent=1)

from datetime import datetime, timedelta

In [2]:
def get_parsed_json(path):
    request = Request(path)
    response = urlopen(request)
    json_rsp = response.read()
    parsed_json = json.loads(json_rsp)
    return parsed_json

In [3]:
def is_reg_season_game(game_data):
    return True if str(game_data['games'][0]['gamePk'])[5] == '2' else False

In [4]:
def is_home_game(game, team_id):
    return True if game['games'][0]['teams']['home']['team']['id'] == team_id else False

In [5]:
def won_game(game):
    return 1 if game['games'][0]['teams']['home']['score'] > game['games'][0]['teams']['away']['score'] else 0

In [6]:
def get_team_api_stats(team_standings, team_id):
    
    team_stats = []
    
    # loop through conferences
    for i in range(6):
        # loop through games played on night
        for j in range(len(team_standings[i]['teamRecords'])):
            team_records = team_standings[i]['teamRecords'][j]
            if team_records['team']['id'] == team_id:
                team_stats.append(team_records['goalsScored'])
                team_stats.append(team_records['goalsAgainst'])
                team_stats.append(team_records['streak']['streakNumber'])
                team_stats.append(int(team_records['divisionRank']))
                team_stats.append(int(team_records['conferenceRank']))
                team_stats.append(int(team_records['leagueRank']))
                return team_stats

In [7]:
def get_team_nhl_stats(team_standings):

    team_stats = []
    
    team_stats.append(team_standings['ppPctg'] * 100)
    team_stats.append(team_standings['penaltyKillPctg'] * 100)
    team_stats.append(round(team_standings['goalsFor'] / float(team_standings['shotsFor']), 2) * 100)
    team_stats.append(round(team_standings['goalsAgainst'] / float(team_standings['shotsAgainst']), 2) * 100)
    team_stats.append(team_standings['faceoffWinPctg'] * 100)
    return team_stats

In [8]:
# Initialize dataframe

features = ['game_date', 'game_result', 'home_team_id','home_goals_for', 'home_goals_against', \
            'home_goal_differential', 'home_winning_streak', 'home_division_standing', \
            'home_conference_standing', 'home_league_standing', 'home_power_play_success_rate', \
            'home_power_kill_success_rate', 'home_shot_percentage', 'home_save_percentage', \
            'home_faceoff_win_percentage', 'home_fenwick_close_percentage', 'home_pdo', \
            'away_team_id','away_goals_for', 'away_goals_against', 'away_goal_differential', \
            'away_winning_streak', 'away_division_standing', 'away_conference_standing', 'away_league_standing', \
            'away_power_play_success_rate', 'away_power_kill_success_rate', 'away_shot_percentage', \
            'away_save_percentage', 'away_faceoff_win_percentage', 'away_fenwick_close_percentage', 'away_pdo']
df = pd.DataFrame(columns=features)
df

,game_date,game_result,home_team_id,home_goals_for,home_goals_against,home_goal_differential,home_winning_streak,home_division_standing,home_conference_standing,home_league_standing,home_power_play_success_rate,home_power_kill_success_rate,home_shot_percentage,home_save_percentage,home_faceoff_win_percentage,home_fenwick_close_percentage,home_pdo,away_team_id,away_goals_for,away_goals_against,away_goal_differential,away_winning_streak,away_division_standing,away_conference_standing,away_league_standing,away_power_play_success_rate,away_power_kill_success_rate,away_shot_percentage,away_save_percentage,away_faceoff_win_percentage,away_fenwick_close_percentage,away_pdo


In [9]:
def build_game_df(game_stat_lst):
    print(game_stat_lst)
    print(len(game_stat_lst))
    
    game_record = ["'{}':game_stat_lst[{}]".format(features[i], i) for i in range(len(features))]
    game_record

    dicti = {i for i in game_record}
    dicti
    
    game_df = pd.DataFrame(dicti, index=[0])
    return game_df

In [10]:
def build_game_df(game_stat_lst):
    game_df = pd.DataFrame({'game_date': game_stat_lst[0],
                            'game_result': game_stat_lst[1],
                            'home_team_id': game_stat_lst[2],
                            'home_goals_for': game_stat_lst[3],
                            'home_goals_against': game_stat_lst[4],
                            'home_goal_differential': game_stat_lst[5],
                            'home_winning_streak': game_stat_lst[6],
                            'home_division_standing': game_stat_lst[7],
                            'home_conference_standing': game_stat_lst[8],
                            'home_league_standing': game_stat_lst[9],
                            'home_power_play_success_rate': game_stat_lst[10],
                            'home_power_kill_success_rate': game_stat_lst[11],
                            'home_shot_percentage': game_stat_lst[12],
                            'home_save_percentage': game_stat_lst[13],
                            'home_faceoff_win_percentage': game_stat_lst[14],
                            'home_fenwick_close_percentage': game_stat_lst[15],
                            'home_pdo': game_stat_lst[16],
                            'away_team_id': game_stat_lst[17],
                            'away_goals_for': game_stat_lst[18],
                            'away_goals_against': game_stat_lst[19],
                            'away_goal_differential': game_stat_lst[20],
                            'away_winning_streak': game_stat_lst[21],
                            'away_division_standing': game_stat_lst[22],
                            'away_conference_standing': game_stat_lst[23],
                            'away_league_standing': game_stat_lst[24],
                            'away_power_play_success_rate': game_stat_lst[25],
                            'away_power_kill_success_rate': game_stat_lst[26],
                            'away_shot_percentage': game_stat_lst[27],
                            'away_save_percentage': game_stat_lst[28],
                            'away_faceoff_win_percentage': game_stat_lst[29],
                            'away_fenwick_close_percentage': game_stat_lst[30],
                            'away_pdo': game_stat_lst[31]},
                            index=[0])
    return game_df

In [ ]:
for i in range(1,31):
    team_path = 'https://statsapi.web.nhl.com/api/v1/schedule?teamId=' + str(i) + '&startDate=2005-10-01&endDate=2018-04-25'
    team_data = get_parsed_json(team_path)
    num_of_games = len(team_data['dates'])

    for j in range(0, num_of_games):
        game_data = team_data['dates'][j]
        
        if is_reg_season_game(game_data) and is_home_game(game_data, i) :
            game_id = game_data['games'][0]['gamePk']
            game_date = game_data['date']
            game_result = won_game(game_data)
            home_team_id = i
            away_team_id = game_data['games'][0]['teams']['away']['team']['id']
            
            # for making calls to API
            if game_date[5] == '1': # month is Oct_Dec
                game_year = game_date[:4]
            else:
                game_year = str(datetime.strptime(game_date, '%Y-%m-%d') - timedelta(days=365))[:4]
            
#             print("game_id: {}".format(game_id))
#             print("game_date: {}".format(game_date))
#             print("game_result: {}".format(game_result))
#             print("home_team_id: {}".format(i))
#             print("away_team_id: {}".format(away_team_id))
#             print("\n")
            
            # for when making API calls since game result isn't available until day after
            # i.e. we don't want stats about the game we are trying to predict
            # e.g. Leafs play on January 1 then we want stats up to and including December 31
            date_after_game = str(datetime.strptime(game_date, '%Y-%m-%d') + timedelta(days=1))[:10]
            
            team_standings_path = 'https://statsapi.web.nhl.com/api/v1/standings?date=' + str(game_date)
            team_standings = get_parsed_json(team_standings_path)['records']
            home_api_stats = get_team_api_stats(team_standings, i)
            home_goals_for = home_api_stats[0]
            home_goals_against = home_api_stats[1]
            home_goal_differential = home_goals_for - home_goals_against
            home_winning_streak = home_api_stats[2]
            home_division_standing = home_api_stats[3]
            home_conference_standing = home_api_stats[4]
            home_league_standing = home_api_stats[5]
            
            home_nhl_stats_path = 'http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=basic&isGame=true&reportName=teamsummary&sort=[{%22property%22:%22points%22,%22direction%22:%22DESC%22},{%22property%22:%22wins%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%22' + game_year + '-09-01%22%20and%20gameDate%3C=%22' + game_date + '%22%20and%20gameTypeId=2%20and%20teamId=' + str(i) + ''
            home_nhl_stats_json = get_parsed_json(home_nhl_stats_path)
            home_nhl_stats = get_team_nhl_stats(home_nhl_stats_json['data'][0])
            home_power_play_success_rate = round(home_nhl_stats[0], 2)
            home_power_kill_success_rate = round(home_nhl_stats[1], 2)
            home_shot_percentage = round(home_nhl_stats[2], 2)
            home_save_percentage = round(home_nhl_stats[3], 2)
            home_faceoff_win_percentage = round(home_nhl_stats[4], 2)

            home_sat_percentages_path = 'http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=shooting&isGame=true&reportName=teampercentages&sort=[{%22property%22:%22shotAttemptsPctg%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%22' + game_year + '-09-01%22%20and%20gameDate%3C=%22' + game_date + '%22%20and%20gameTypeId=2%20and%20teamId=' + str(i) + ''
            home_sat_percentages = get_parsed_json(home_sat_percentages_path)['data'][0]
            home_fenwick_close_percentage = round(home_sat_percentages['unblockedShotAttemptsPctgClose'] * 100, 2)
            home_pdo = round((home_sat_percentages['shootingPlusSavePctg']) * 1000, 2)

#             print("home_goals_for: {}".format(home_goals_for))
#             print("home_goals_against: {}".format(home_goals_against))
#             print("home_goal_differential: {}".format(home_goal_differential))
#             print("home_winning_streak: {}".format(home_winning_streak))
#             print("home_division_standing: {}".format(home_division_standing))
#             print("home_conference_standing: {}".format(home_conference_standing))
#             print("home_league_standing: {}".format(home_league_standing))
#             print("home_power_play_success_rate: {}".format(home_power_play_success_rate))
#             print("home_power_kill_success_rate: {}".format(home_power_kill_success_rate))
#             print("home_shot_percentage: {}".format(home_shot_percentage))
#             print("home_save_percentage: {}".format(home_save_percentage))
#             print("home_faceoff_win_percentage: {}".format(home_faceoff_win_percentage))
#             print("home_fenwick_close_percentage: {}".format(home_fenwick_close_percentage))
#             print("home_pdo: {}".format(home_pdo))
#             print("\n")

            away_api_stats = get_team_api_stats(team_standings, away_team_id)
            away_goals_for = away_api_stats[0]
            away_goals_against = away_api_stats[1]
            away_goal_differential = away_goals_for - away_goals_against
            away_winning_streak = away_api_stats[2]
            away_division_standing = away_api_stats[3]
            away_conference_standing = away_api_stats[4]
            away_league_standing = away_api_stats[5]
                     
            away_nhl_stats_path = 'http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=basic&isGame=true&reportName=teamsummary&sort=[{%22property%22:%22points%22,%22direction%22:%22DESC%22},{%22property%22:%22wins%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%22' + game_year + '-09-01%22%20and%20gameDate%3C=%22' + game_date + '%22%20and%20gameTypeId=2%20and%20teamId=' + str(away_team_id) + ''
            away_nhl_stats_json = get_parsed_json(away_nhl_stats_path)
            away_nhl_stats = get_team_nhl_stats(away_nhl_stats_json['data'][0])
            away_power_play_success_rate = round(away_nhl_stats[0], 2)
            away_power_kill_success_rate = round(away_nhl_stats[1], 2)
            away_shot_percentage = round(away_nhl_stats[2], 2)
            away_save_percentage = round(away_nhl_stats[3], 2)
            away_faceoff_win_percentage = round(away_nhl_stats[4], 2)
            
            away_sat_percentages_path = 'http://www.nhl.com/stats/rest/team?isAggregate=true&reportType=shooting&isGame=true&reportName=teampercentages&sort=[{%22property%22:%22shotAttemptsPctg%22,%22direction%22:%22DESC%22}]&factCayenneExp=gamesPlayed%3E=1&cayenneExp=gameDate%3E=%22' + game_year + '-09-01%22%20and%20gameDate%3C=%22' + game_date + '%22%20and%20gameTypeId=2%20and%20teamId=' + str(away_team_id) + ''
            away_sat_percentages = get_parsed_json(away_sat_percentages_path)['data'][0]
            away_fenwick_close_percentage = round(away_sat_percentages['unblockedShotAttemptsPctgClose'] * 100, 2)
            away_pdo = round((away_sat_percentages['shootingPlusSavePctg']) * 1000, 2)

#             print("away_goals_for: {}".format(away_goals_for))
#             print("away_goals_against: {}".format(away_goals_against))
#             print("away_goal_differential: {}".format(away_goal_differential))
#             print("away_winning_streak: {}".format(away_winning_streak))
#             print("away_division_standing: {}".format(away_division_standing))
#             print("away_conference_standing: {}".format(away_conference_standing))
#             print("away_league_standing: {}".format(away_league_standing))
#             print("away_power_play_success_rate: {}".format(away_power_play_success_rate))
#             print("away_power_kill_success_rate: {}".format(away_power_kill_success_rate))
#             print("away_shot_percentage: {}".format(away_shot_percentage))
#             print("away_save_percentage: {}".format(away_save_percentage))
#             print("away_faceoff_win_percentage: {}".format(away_faceoff_win_percentage))
#             print("away_fenwick_close_percentage: {}".format(away_fenwick_close_percentage))
#             print("away_pdo: {}".format(away_pdo))
#             print("\n------------------------------------------\n")

            game_stat_lst = [game_date, game_result, i, home_goals_for, home_goals_against, home_goal_differential, \
                             home_winning_streak, home_division_standing, home_conference_standing, home_league_standing, \
                             home_power_play_success_rate, home_power_kill_success_rate, home_shot_percentage, \
                             home_save_percentage, home_faceoff_win_percentage, home_fenwick_close_percentage, home_pdo, \
                             away_team_id, away_goals_for, away_goals_against, away_goal_differential, \
                             away_winning_streak, away_division_standing, away_conference_standing, away_league_standing, \
                             away_power_play_success_rate, away_power_kill_success_rate, away_shot_percentage, \
                             away_save_percentage, away_faceoff_win_percentage, away_fenwick_close_percentage, away_pdo]

            game_df = build_game_df(game_stat_lst)

            df = df.append(game_df, ignore_index=True)

In [ ]:
# To get last three games, first pull the scehdule of the team and then the corresponding stats
# Create a dictionary of dates and stats. then write a function to pull the last X days game stats